In [1]:
import tensorflow as tf
import os
from scipy.signal import resample, butter, sosfilt
from scipy.io import wavfile
import numpy as np
import random

In [2]:
tf.keras.utils.set_random_seed(17039)

In [3]:
dataset_folder = "dataset_full"
train = 0.8
val = 0.1
test = 0.1
version = 2

In [4]:
folder_list = os.listdir(f".\\datasets\\raw\\{dataset_folder}")
x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []
filter = butter(4, (20, 150), btype='bandpass', output='sos', fs=8000)
for idx, folder in enumerate(folder_list):
    file_names = os.listdir(f".\\datasets\\raw\\{dataset_folder}\\{folder}")
    no_of_files = len(file_names)
    y_train.extend([idx] * int(no_of_files * train))
    y_val.extend([idx] * int(no_of_files * val))
    y_test.extend([idx] * int(no_of_files * test))
    x = []
    for file in file_names:
        _, raw_signal = wavfile.read(f".\\datasets\\raw\\{dataset_folder}\\{folder}\\{file}")
        preprocessed_signal = (raw_signal / np.max(np.abs(raw_signal))) * 32767
        preprocessed_signal = sosfilt(filter, preprocessed_signal)
        preprocessed_signal = resample(preprocessed_signal, 20000)
        x.append(np.reshape(preprocessed_signal, (20000, 1)))
    random.shuffle(x)
    x_train.extend(x[:int(no_of_files * train)])
    x_val.extend(x[int(no_of_files * train):int(no_of_files * (train + val))])
    x_test.extend(x[int(no_of_files * (train + val)):])
y_train = tf.one_hot(y_train, 5)
y_val = tf.one_hot(y_val, 5)
y_test = tf.one_hot(y_test, 5)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [7]:
os.mkdir(f".\\datasets\\{dataset_folder}_version_{version}")
train_dataset.save(f".\\datasets\\{dataset_folder}_version_{version}\\train_dataset")
val_dataset.save(f".\\datasets\\{dataset_folder}_version_{version}\\val_dataset")
test_dataset.save(f".\\datasets\\{dataset_folder}_version_{version}\\test_dataset")